In [ ]:
l1_cat_name_lower = ''

In [2]:
import os
import sys
user_name = os.path.dirname(os.path.realpath(sys.argv[0])).split('/')[2]
user_home = f'/home/{user_name}'
user_home

'/home/apple'

In [ ]:
import importlib

l1_cat_lc = l1_cat_name_lower
sys.path.insert(1, f'user_home')

# from roberta_l2_unpack_wip_new import load_ckp, RoBERTaClass, CustomDataset
# from roberta_l2_unpack_wip_new import RoBERTaClass, CustomDataset

RoBERTaClass = getattr(importlib.import_module(f"roberta_clt_{l1_cat_lc}_training"), 'RoBERTaClass')
CustomDataset = getattr(importlib.import_module(f"roberta_clt_{l1_cat_lc}_training"), 'CustomDataset')

In [ ]:
processed_dir_path=f""
validation_case_level_path = f""


In [ ]:
import nltk

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

import regex as re
import torch
import pandas as pd
import numpy as np


from transformers import RobertaTokenizer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def preprocess(text):
    
    text = re.sub('[0-9]', '', text)                  # remove numbers  
    text = re.sub(' +', ' ', text)                    # remove multiple whitespaces
    text = text.replace("'", "")                      # remove apostrophe
    text = re.sub('[-!,;+"%>|<@:#_=$£/*]', ' ', text) # remove special characters 
    text = re.sub(r"[\([{})\]]", " ", text)           # remove brackets 
    text = text.replace("\n", "." )                   # replace newline with .
    text = text.replace("\t", "" )                    # replace '?' with whitespace
    text = text.replace('?', ' ') 
    text = text.replace('e.g.', ' ')                  # remove elements like e.g., i.e., etc.
    text = text.replace('i.e.', ' ')
    text = text.replace('etc.', ' ') 
    text = text.replace("x000D", "." )  
    text = text.replace("Seperator", "." ) 
    text = text.lower()                               # convert all text to lowercase
    text = nltk.sent_tokenize(text) 
    
    return text

In [ ]:
train_sentence_level_path = f""
df_train = pd.read_csv(train_sentence_level_path,index_col=False)
if 'Unnamed: 0' in df_train.columns:
    df_train=df_train.drop('Unnamed: 0',axis=1)
# sorted(df_train['label'].unique().tolist())
target_list = sorted(df_train['label'].value_counts().index)
target_list

In [ ]:

df_train['label'].value_counts()    

In [ ]:
df_val = pd.read_csv(validation_case_level_path, index_col=False)
if 'Unnamed: 0' in df_val.columns:
    df_val=df_val.drop('Unnamed: 0',axis=1)
df_val

In [ ]:
(
    df_val[''].value_counts(),
    sum(df_val[''].value_counts())
)

In [ ]:
df_val.shape

In [ ]:
text_label_list=[]
case_category_list=[]
for index in range(df_val.shape[0]):
    label = 'noise'
    l1_cat_string = ''
   # l2_cat_string = 'Contact reason 2nd level'
    row=df_val.iloc[index]
    case_number=str(row[''])
    text = str(row[''])
    text += str(row['']) 
    text = text + str(row[''])
    text = text + str(row[''])
    text = text + str(row[''])
    
    # text = str(row['Trigger sentences'])
    # if (row[l1_cat_string] == l1_category_name):
    label = row[l1_cat_string]
    text_label_list.append({'case_number':case_number, 'text': text, 'label': label})
df_val_v1=pd.DataFrame(text_label_list)
df_val_v1.head()

In [ ]:
#df_val_v1 = df_val_v1.loc[:, ['text', 'label']]
df_val_v1.dropna(subset=['text'], inplace=True)
df_val_v1 = df_val_v1.reset_index(drop=True)
df_val_v1[189:290]

In [ ]:
def maxindex(data_list):
    index=0
    for i in range(len(data_list)):
        if (data_list[i]>data_list[index]):
            index=i
    return index

In [ ]:
train_sentence_level_path = f""
df_train = pd.read_csv(train_sentence_level_path, index_col=False)
df_train=df_train.reset_index(drop=True)
df_train['label'].value_counts() 

In [ ]:

target_list = sorted(df_train['label'].value_counts().index)
import gc
del df_train
gc.collect()

target_list

In [ ]:
# hyperparameters
MAX_LEN = 256
# TRAIN_BATCH_SIZE = 8
# VALID_BATCH_SIZE = 8
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 15
LEARNING_RATE = 1e-05

In [ ]:
def make_tagged_dataset(df_val_v1):
    case_tags_val = pd.get_dummies(df_val_v1.label, dtype=int)
    case_tags_val_columns = case_tags_val.columns.tolist()
    # to add missing columns in validation dataframe 
    for column in target_list:
        if column not in case_tags_val_columns:
            case_tags_val[column]=pd.Series(0,index=range(case_tags_val.shape[0]))
    case_tags_val_columns = sorted(case_tags_val.columns.tolist())
    case_tags_val=case_tags_val[case_tags_val_columns]

    df_val_v2 = pd.concat([df_val_v1, case_tags_val], axis=1)
    return df_val_v2

In [ ]:
def output_case_category(model, device, case_data_loader):
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)
    
    output_categories=[]
    output_list=[]
    for batch_idx, data in enumerate(case_data_loader):
        #print('yyy epoch', batch_idx)
        # print(data['input_ids'])
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        for output in outputs:
            output_list.append(output)
        print(output_list)
    return output_list

In [ ]:
df_val_v1.head()

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model

In [ ]:
# noise_index = target_list.index('noise')
# print(noise_index)

In [ ]:
# validation_case_level_path = f"{processed_dir_path}/Common/val_cases.csv"
val_df = pd.read_csv(validation_case_level_path)
val_df

In [ ]:
# import nltk
# nltk.download("punkt")
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

In [ ]:
target_list

In [ ]:
########## Reevu's code snippet refactored to populate confusion matrix
#######################################################################


# target_cols = ['availability', 'compatibility_of_parts', 'features/benefits', 'noise', 'order_product', 'order_status', 'price',
#                'product_information']


target_cols = target_list

output_list = []

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = RoBERTaClass()
model.to(device)

optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)

model=load_ckp(best_model_path,model,optimizer)

confusion_matrix = pd.DataFrame(0,index=target_list,columns=target_list)

TP=0
FP=0
TN=0
FN=0

In [8]:
dummy_target_list=[]
for i in range(0,8):
    dummy_target_list += [f"C{i+1}"]
dummy_target_list

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']

In [9]:
confidence_intervals=['<1%',
                      '1%-10%',
                      '10%-20%',
                      '20%-30%',
                      '30%-40%',
                      '40%-50%',
                      '50%-60%',
                      '60%-70%',
                      '70%-80%',
                      '80%-90%',
                      '90%-100%']

#success_confidence_matrix= pd.DataFrame(0,index=target_list,columns=confidence_intervals)
success_confidence_matrix= pd.DataFrame(0,index=dummy_target_list,columns=confidence_intervals)
success_confidence_matrix

,<1%,1%-10%,10%-20%,20%-30%,30%-40%,40%-50%,50%-60%,60%-70%,70%-80%,80%-90%,90%-100%
C1,0,0,0,0,0,0,0,0,0,0,0
C2,0,0,0,0,0,0,0,0,0,0,0
C3,0,0,0,0,0,0,0,0,0,0,0
C4,0,0,0,0,0,0,0,0,0,0,0
C5,0,0,0,0,0,0,0,0,0,0,0
C6,0,0,0,0,0,0,0,0,0,0,0
C7,0,0,0,0,0,0,0,0,0,0,0
C8,0,0,0,0,0,0,0,0,0,0,0


In [10]:

#failure_confidence_matrix= pd.DataFrame(0,index=target_list,columns=confidence_intervals)
failure_confidence_matrix= pd.DataFrame(0,index=dummy_target_list,columns=confidence_intervals)
failure_confidence_matrix

,<1%,1%-10%,10%-20%,20%-30%,30%-40%,40%-50%,50%-60%,60%-70%,70%-80%,80%-90%,90%-100%
C1,0,0,0,0,0,0,0,0,0,0,0
C2,0,0,0,0,0,0,0,0,0,0,0
C3,0,0,0,0,0,0,0,0,0,0,0
C4,0,0,0,0,0,0,0,0,0,0,0
C5,0,0,0,0,0,0,0,0,0,0,0
C6,0,0,0,0,0,0,0,0,0,0,0
C7,0,0,0,0,0,0,0,0,0,0,0
C8,0,0,0,0,0,0,0,0,0,0,0


In [ ]:

accuracy_confidence_matrix= pd.DataFrame(0.0,index=target_list,columns=confidence_intervals)
accuracy_confidence_matrix

In [ ]:
import math

In [ ]:
def get_index_matching_confidence_score(conf_score):
    if(conf_score<=0.01):
        return 0
    else:
        return math.ceil(conf_score*10.0)
    
get_index_matching_confidence_score(1.0)

In [ ]:
model.linear

In [ ]:
tokenizer

In [ ]:
ct = 0
for idx in range(val_df.shape[0]):
    text=[]
    
    text += preprocess( str(val_df[''].iloc[idx]) )
    text += preprocess( str(val_df[''].iloc[idx]) )
    text += preprocess( str(val_df[''].iloc[idx]) )
    text += preprocess( str(val_df[''].iloc[idx]) )

    ### get the ground truth label
    ground_truth = val_df[''].iloc[idx]
    if ground_truth not in target_list:
        print(f"{ground_truth} not found in target list")
        ground_truth = 'noise'


    ### process each sentence in text
    prediction_text = {}
    
    txt=''
    for element in text:
        txt += element
        
    encodings = tokenizer.encode_plus(
        txt,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    model.eval()
    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()

        curr_conf_score = max(final_output[0])
        pred_label = target_cols[int(np.argmax(final_output, axis=1))]

        if pred_label:
            if not pred_label == 'noise':
                if pred_label in prediction_text:
                    if prediction_text[pred_label] < curr_conf_score:
                        prediction_text[pred_label] = curr_conf_score
                else:
                    prediction_text[pred_label] = curr_conf_score  

    
    ### CORNER CASE --- IF MODEL DOES NOT PREDICT ANY LEVEL 1 CATEGORY, THEN SET A 
    ### DEFAULT CATEGORY AS 'UNCLEAR' --- THIS WILL MAKE SURE CODE WILL NOT BREAK
    if len(list(prediction_text.keys())):
        predicted_label = max(prediction_text, key=prediction_text.get) 
    else:
        predicted_label = 'noise'
    
    output_list += (prediction_text, predicted_label, ground_truth, idx)
    
    if predicted_label == ground_truth:
        ct += 1
        if ground_truth == 'noise':
            TN += 1
        else:
            TP += 1
    else:
        if ground_truth == 'noise':
            FP += 1
        else:
            FN += 1

    
    confusion_matrix[ground_truth][predicted_label] += 1
    
    l1_softmax_preds=final_output[0]
    
    for cat_index in range(len(l1_softmax_preds)):
        conf_score=l1_softmax_preds[cat_index]
        current_cat_name=target_list[cat_index]
        interval_index=get_index_matching_confidence_score(conf_score)
        matrix_to_increment = success_confidence_matrix if (current_cat_name == ground_truth) else failure_confidence_matrix
        conf_interval_col_name=matrix_to_increment.columns[interval_index]
        matrix_to_increment.loc[current_cat_name,conf_interval_col_name] += 1

print(ct)
#print(output_list)

In [ ]:
# success_confidence_matrix=success_confidence_matrix.rename(columns={'10%-20':'10%-20%', '20%-30':'20%-30%' })
success_confidence_matrix

In [ ]:
# failure_confidence_matrix=failure_confidence_matrix.rename(columns={'10%-20':'10%-20%', '20%-30':'20%-30%' })
failure_confidence_matrix

In [ ]:
ct/val_df.shape[0]

In [ ]:
ct

In [ ]:
val_df.shape[0]

In [ ]:
(TP*100.0)/(TP+FN)

In [ ]:
confusion_matrix

In [ ]:
import matplotlib
import seaborn as sns

def save_df_as_image(df, path):
    # Make plot
    plot = sns.heatmap(df,
                       annot=True,
                       cmap="Blues",
                       linewidth=0.5,
                       cbar=False,
                       robust=True,
                       fmt="d")
    plot.set_xlabel('Ground Truth')
    plot.set_ylabel('Predicted Category')
    fig = plot.get_figure()
    fig.savefig(path,bbox_inches="tight")

In [ ]:
save_df_as_image(confusion_matrix,"confusion_matrix_roberta_l1_cat_clt.png")

In [ ]:
precision = (TP * 100.0)/(TP + FP)
recall = (TP * 100.0)/(TP + FN)
{"Precision":precision, "Recall":recall}

In [ ]:
precisions=pd.Series(0.0,index=confusion_matrix.index)
recalls=pd.Series(0.0,index=confusion_matrix.index)
for index in precisions.index:
    precisions[index]=(confusion_matrix[index][index]*100.0)/sum(confusion_matrix.loc[index])
    recalls[index]=(confusion_matrix[index][index])*100.0/sum(confusion_matrix[index])

In [ ]:
precisions

In [ ]:
recalls

In [ ]:
metrics=pd.DataFrame(0,index=confusion_matrix.index,columns=['precision','recall'])
metrics['precision']=precisions
metrics['recall']=recalls
metrics

In [ ]:
# accuracy_confidence_matrix=accuracy_confidence_matrix.rename(columns={'10%-20':'10%-20%', '20%-30':'20%-30%' })
for column in accuracy_confidence_matrix.columns:
    for index in accuracy_confidence_matrix.index:
        numerator = success_confidence_matrix[column][index]
        denominator = failure_confidence_matrix[column][index] + numerator
        if denominator == 0:
            accuracy_confidence_matrix.loc[index,column] = np.NINF
        else:
            accuracy_confidence_matrix.loc[index,column]=float(numerator)/denominator

accuracy_confidence_matrix

In [ ]:
metaconfidence_row_name='metaconfidence'
overall_metaconfidence=pd.DataFrame(0.0,index=[metaconfidence_row_name],columns=accuracy_confidence_matrix.columns)
overall_metaconfidence

In [ ]:
for column in success_confidence_matrix:
    addendum_list=success_confidence_matrix[column].tolist()
    for idx in range(len(addendum_list)):
        if addendum_list[idx]==np.NINF:
            addendum_list[idx]=0
    numerator=sum(addendum_list)
    addendum_list=failure_confidence_matrix[column].tolist()
    for idx in range(len(addendum_list)):
        if addendum_list[idx]==np.NINF:
            addendum_list[idx]=0
    denominator=sum(addendum_list) + numerator
    if denominator == 0:
        overall_metaconfidence.loc[metaconfidence_row_name,column]=np.NINF
    else:
        overall_metaconfidence.loc[metaconfidence_row_name,column]=float(numerator*100)/denominator
overall_metaconfidence

In [ ]:
# cat_wise_metrics = ['misclassifications','matches','category_cases','accuracy']
# cat_wise_acc = pd.DataFrame(None,index=multiclass_target_list,columns=cat_wise_metrics)
# cat_wise_acc
# for cat in multiclass_target_list: 
#     cat_wise_acc.loc[cat,'category_cases'] = sum(confusion_matrix[cat])
#     cat_wise_acc.loc[cat,'matches'] = confusion_matrix[cat][cat]
#     cat_wise_acc.loc[cat,'misclassifications'] = sum(confusion_matrix[cat]) - confusion_matrix[cat][cat]
#     cat_wise_acc.loc[cat,'accuracy'] = (cat_wise_acc.loc[cat,'matches'] * 100.0) / cat_wise_acc.loc[cat,'category_cases']
# cat_wise_acc